In [1]:
using Pkg
Pkg.activate("D://ATIS3//Project.toml") # change path 
Pkg.instantiate()

 Activating environment at `D:\ATIS3\Project.toml`


In [2]:
using JuMP, LinearAlgebra, Plots, StatsPlots, CPLEX, Statistics, CSV, DataFrames

## Define parameter

In [42]:
#read scenario data
scenarios_hexagons = CSV.read("scenarios_multinomial.csv", DataFrame)
scenarios_prices = CSV.read("priceScenarions3.csv", DataFrame)

hexagons = names(scenarios_hexagons)[1:length(names(scenarios_hexagons))-1] # vector of hexagons (strings)
hours = names(scenarios_prices)[3:length(names(scenarios_prices))] #vector of hours for charging (strings)
car_kWh = 30 # capacity of each car

#demand mxn (m=hexagon, n=scenario) in kWh summe über car*max_kWh*(1-SOC)
SOC = ones(length(hexagons),6).*rand(Float64, (length(hexagons), 6))
demand_cars = transpose(convert(Matrix,scenarios_hexagons[:,hexagons]))
demand_kWh = demand_cars.*car_kWh.*(ones(length(hexagons),6)-SOC)

#prices mxn (m=hours, n=scenario)
prices = transpose(convert(Matrix,scenarios_prices[:,hours]))

probability_hexagons = transpose(scenarios_hexagons.probs) # probabilities for each scenario (#cars per hexagon)
probability_prices = transpose(scenarios_prices.Probability) # probabilities for each scenario (timeseries prices)

P = [3.7, 11, 22] # different charger types in kW
charger_cost_fix = [1500, 1700, 1900] # investment cost for charger 3 kW, 7 kW und 11 kW in €
charger_cost_annuity = charger_cost_fix./(3*365)
charger_cost = charger_cost_annuity./P # investment cost for charger 3 kW, 7 kW und 11 kW in €/kW
opportunity_cost = 0.4 # cost of demand not served

alpha = 0.05
beta = 0


0

## Build the scenario tree

In [43]:
## Build Probabilities
comb_probability = []
for prob_d in probability_hexagons
    for prob_p in probability_prices
        push!(comb_probability,prob_d*prob_p)
    end
end
comb_probability = convert(Array{Float64,1}, comb_probability)

## Build Demand kWh and Demand Cars
num_hex = size(demand_kWh,1)
num_Scen_Dem = size(demand_kWh,2)
num_Scen_Pri = size(prices,2)

comb_demand_kWh = zeros((num_hex, num_Scen_Dem*num_Scen_Pri))
comb_demand_cars = zeros((num_hex, num_Scen_Dem*num_Scen_Pri))
comb_SOC = zeros((num_hex, num_Scen_Dem*num_Scen_Pri))
counter = 0

for num_d in 1:num_Scen_Dem
    for num_p in 1:num_Scen_Pri
        counter += 1
        comb_demand_kWh[:,counter] = demand_kWh[:,num_d]
        comb_demand_cars[:,counter] = demand_cars[:,num_d]
        comb_SOC[:,counter] = SOC[:,num_d]
    end
end

## Build Prices
num_Scen_Dem = size(demand_kWh,2)
num_Scen_Pri = size(prices,2)

comb_prices = zeros((length(hours), num_Scen_Dem*num_Scen_Pri))
counter = 0

for num_d in 1:num_Scen_Dem
    for num_p in 1:num_Scen_Pri
        counter += 1
        comb_prices[:,counter] = prices[1:length(hours),num_p]
    end
end


97×42 Array{Float64,2}:
 0.0214414  0.0214414  0.0214414  …  0.648296   0.648296   0.648296
 0.843514   0.843514   0.843514      0.112909   0.112909   0.112909
 0.248586   0.248586   0.248586      0.240489   0.240489   0.240489
 0.858583   0.858583   0.858583      0.0852635  0.0852635  0.0852635
 0.25874    0.25874    0.25874       0.692119   0.692119   0.692119
 0.891436   0.891436   0.891436   …  0.739931   0.739931   0.739931
 0.130324   0.130324   0.130324      0.686755   0.686755   0.686755
 0.872223   0.872223   0.872223      0.693702   0.693702   0.693702
 0.754616   0.754616   0.754616      0.613494   0.613494   0.613494
 0.661596   0.661596   0.661596      0.0999086  0.0999086  0.0999086
 0.653203   0.653203   0.653203   …  0.570871   0.570871   0.570871
 0.881498   0.881498   0.881498      0.20706    0.20706    0.20706
 0.898713   0.898713   0.898713      0.0350224  0.0350224  0.0350224
 ⋮                                ⋱             ⋮          
 0.424227   0.424227   0.42422

## Build model

In [44]:
function charger_siting(comb_SOC, hexagons, hours, car_kWh, comb_demand_cars, comb_demand_kWh, comb_prices, 
        comb_probability, P, charger_cost, opportunity_cost, alpha, beta)
    
    opprtunity_cost = 0
    
    # Define Sets
    Γ = Array{Int}(1:length(hexagons)) # Gamma, Set with hexagons
    Σ = Array{Int}(1:length(P)) # Sigma, Set with charger types 
    Ω = Array{Int}(1:length(comb_probability)) # Omega, Set with scenarios
    Τ = Array{Int}(1:length(hours)) # Lambda, Set with scenarios for prices

    # Initialize model    
    m = Model(CPLEX.Optimizer)
    
    # η
    @variable(m, η) 
    @variable(m, 0 <= s[k=Ω])
    β = beta 
    α = alpha
    
    # Define variables
    @variable(m, 0 <= x_charger[i=Γ,j=Σ],Int) # number of specific charger type in each hexagon
    @variable(m, 0 <= E_supplied[i=Γ,j=Σ,k=Ω,t=Τ]) # supplied energy in each hexagon kWh
    @variable(m, 0 <= E_not_supplied[i=Γ,k=Ω])     # demand not covered kWh
    @variable(m, 0 <= cars_supplied[i=Γ,j=Σ,k=Ω],Int)
    @variable(m, 0 <= cars_not_supplied[i=Γ,k=Ω],Int)

    # expressions for total costs
    @expression(m, invest_cost[i=Γ,j=Σ], charger_cost[j].*x_charger[i,j].*P[j]) # first stage investment cost
    @expression(m, charging_opportunity_cost[i=Γ,k=Ω], opportunity_cost.*E_not_supplied[i,k].*comb_probability[k])
    # @expression(m, grid_usage[i=Γ,k=Ω,t=Τ], 0.1*sum(E_supplied[i,j,k,t] for j in Σ)^(2).*comb_probability[k])
    @expression(m, grid_usage[i=Γ,k=Ω,t=Τ], 0.1*sum(E_supplied[i,j,k,t] for j in Σ).*comb_probability[k])
    @expression(m, charging_cost[i=Γ,j=Σ,k=Ω,t=Τ], comb_prices[t,k].*E_supplied[i,j,k,t].*comb_probability[k]) #minimize cost of charging
    
    # expressions for objective function
    @expression(m, total_cost, (-1)*(  sum(invest_cost) 
                                     + sum(charging_cost)
                                     + sum(charging_opportunity_cost) 
                                     + sum(grid_usage)
                                     )
                )
    @expression(m, cVAR, η-(1/(1-α) * dot(comb_probability, s)))
    @objective(m, Max, (1-β)*total_cost + β*cVAR)
    
    # Constraints
    # Eq. Balance equation: Demand in each hexagon has to equal demand. Loss of load is possible
    @constraint(m, eq_balance[i=Γ,k=Ω], sum(E_supplied[i,j,k,t] for j in Σ for t in Τ) + E_not_supplied[i,k] == comb_demand_kWh[i,k])

    # Eq. energy limit: rated_power*chargingtime must not exeed energy
    @constraint(m, eq_cap[i=Γ,j=Σ,k=Ω,t=Τ], E_supplied[i,j,k,t] <= x_charger[i,j].*P[j])

    # Eq. supplied cars and not supplied cars have to equal amount of cars in each hexagon
    @constraint(m, eq_cars_supplied[i=Γ,k=Ω],     sum(cars_supplied[i,j,k] for j in Σ) 
                                                    + cars_not_supplied[i,k]
                                                   == comb_demand_cars[i,k])

    # Eq. maximum one car per charging station
    @constraint(m, eq_max_cars[i=Γ,j=Σ,k=Ω], cars_supplied[i,j,k] <= x_charger[i,j])

    # Eq. 
    @constraint(m, eq_max_supply[i=Γ,j=Σ,k=Ω], sum(E_supplied[i,j,k,t] for t in Τ) == cars_supplied[i,j,k].*car_kWh.*(1-comb_SOC[i,k]))

    # η Constraint
    @constraint(m, cVAR_constraint[k=Ω], η + (sum((charger_cost[j].*x_charger[i,j].*P[j]) for j in Σ for i in Γ) 
                                           +  sum((opportunity_cost.*E_not_supplied[i,k]) for i in Γ)
                                           # +  sum(0.1*sum(E_supplied[i,j,k,t] for j in Σ)^(2) for i in Γ for t in Τ)
                                           +  0.1*sum(E_supplied[i,j,k,t] for j in Σ for i in Γ for t in Τ)
                                           +  sum((comb_prices[t,k].*E_supplied[i,j,k,t]) for j in Σ for i in Γ for t in Τ))
                                           <= s[k])

    # Run optimization
    optimize!(m)

    # Evaluate resuluts
    obj = objective_value(m)
    charger_in_hex = value.(x_charger)
    
    cost1 = value.(sum(invest_cost)) 
    cost2 = value.(sum(charging_cost))
    cost3 = value.(sum(charging_opportunity_cost)) 
    cost4 = value.(sum(grid_usage))
    
    println("cost invest", cost1)
    println("cost charging",cost2)
    println("cost opportunity",cost3)
    println("cost grid",cost4)
    
    res_total_cost = value.(total_cost)
    cvar = value.(cVAR)
    
    return charger_in_hex, E_not_supplied, res_total_cost, cvar
end

charger_siting (generic function with 1 method)

## Run model base case 

In [45]:
charger_in_hex, E_not_supplied, res_total_cost, cvar = charger_siting(
    comb_SOC, hexagons, hours, car_kWh, comb_demand_cars, comb_demand_kWh, comb_prices, 
        comb_probability, P, charger_cost, opportunity_cost, alpha, beta)

println(round.(charger_in_hex))
println("cvar:",cvar)
println("total_cost:",res_total_cost)

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
Found incumbent of value -3065.207957 after 0.02 sec. (13.97 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 39424 rows and 35414 columns.
MIP Presolve modified 8085 coefficients.
Aggregator did 1911 substitutions.
Reduced MIP has 76853 rows, 68933 columns, and 266168 nonzeros.
Reduced MIP has 1578 binaries, 7603 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.23 sec. (190.20 ticks)
Probing time = 0.03 sec. (4.62 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 2562 rows and 2313 columns.
Reduced MIP has 74291 rows, 66620 columns, and 257250 nonzeros.
Reduced MIP has 1331 binaries, 7539 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.17 sec. (128.82 ticks)
Probing time = 0.05 sec. (4.67 ticks)
Clique table members: 434.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution

## Run Sensitivity analysis 

### Opportunity cost 

In [48]:
opp_list = [0.10 0.20 0.30 0.40 0.50 0.60 0.70 0.80 0.90 1.00]
charger_per_type_dict = Dict()
for opportunity_cost in opp_list
    charger_in_hex, E_not_supplied, res_total_cost, cvar = charger_siting(
        comb_SOC, hexagons, hours, car_kWh, comb_demand_cars, comb_demand_kWh, comb_prices,
        comb_probability, P, charger_cost, opportunity_cost, alpha, beta)
    
    I, J = axes(charger_in_hex)
    xx = zeros(length(I),length(J))
    [xx[i,j] = charger_in_hex[i,j] for i in I, j in J]
    charger_per_type = sum(xx, dims=1)
    charger_per_type_dict[string(opportunity_cost)] = charger_per_type
    
end

charger_per_type_dict

auswertung = []
for i in opp_list
    values = charger_per_type_dict[string(i)]
    push!(auswertung, values)
end
auswertung

auswertung3 = vcat(auswertung...)

plot(opp_list',auswertung3, xlabel="Opportunity costs [€/kWh]", ylabel="total number of charging stations",label=["3.7 kW" "11 kW" "22 kW"])

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
Found incumbent of value -766.301989 after 0.02 sec. (13.97 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 39424 rows and 35414 columns.
MIP Presolve modified 8085 coefficients.
Aggregator did 1911 substitutions.
Reduced MIP has 76853 rows, 68933 columns, and 266168 nonzeros.
Reduced MIP has 1578 binaries, 7603 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.23 sec. (190.20 ticks)
Probing time = 0.03 sec. (4.62 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 2562 rows and 2313 columns.
Reduced MIP has 74291 rows, 66620 columns, and 257250 nonzeros.
Reduced MIP has 1331 binaries, 7539 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.19 sec. (128.82 ticks)
Probing time = 0.05 sec. (4.67 ticks)
Clique table members: 434.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution 

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
Found incumbent of value -3831.509947 after 0.02 sec. (13.97 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 39424 rows and 35414 columns.
MIP Presolve modified 8085 coefficients.
Aggregator did 1911 substitutions.
Reduced MIP has 76853 rows, 68933 columns, and 266168 nonzeros.
Reduced MIP has 1578 binaries, 7603 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.27 sec. (190.20 ticks)
Probing time = 0.05 sec. (4.62 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 2562 rows and 2313 columns.
Reduced MIP has 74291 rows, 66620 columns, and 257250 nonzeros.
Reduced MIP has 1331 binaries, 7539 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.20 sec. (128.82 ticks)
Probing time = 0.06 sec. (4.67 ticks)
Clique table members: 434.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
Found incumbent of value -6896.717904 after 0.02 sec. (13.97 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 39424 rows and 35414 columns.
MIP Presolve modified 8085 coefficients.
Aggregator did 1911 substitutions.
Reduced MIP has 76853 rows, 68933 columns, and 266168 nonzeros.
Reduced MIP has 1578 binaries, 7603 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.25 sec. (190.20 ticks)
Probing time = 0.03 sec. (4.62 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 2562 rows and 2313 columns.
Reduced MIP has 74291 rows, 66620 columns, and 257250 nonzeros.
Reduced MIP has 1331 binaries, 7539 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.20 sec. (128.82 ticks)
Probing time = 0.05 sec. (4.67 ticks)
Clique table members: 434.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution

10-element Array{Any,1}:
 [4.0 105.0 639.0]
 [4.0 105.0 639.0]
 [4.0 103.0 641.0]
 [3.0 104.0 640.0]
 [3.0 106.0 639.0]
 [3.0 109.0 637.0]
 [4.0 106.0 639.0]
 [4.0 106.0 639.0]
 [4.0 106.0 639.0]
 [4.0 106.0 639.0]

### Charging hours 

In [ ]:
hours_lst = [["22", "23", "24", "1", "2", "3", "4"],
            ["23", "24", "1", "2", "3", "4"],
            ["24", "1", "2", "3", "4"],
            ["1", "2", "3", "4"],
            ["2", "3", "4"],
            ["3", "4"]]
charger_hours_dict = Dict()
energy_hours_dict = Dict()
cost_hours_dict = Dict()
cvar_hours_dict = Dict()
for hours in hours_lst
    charger_in_hex, E_not_supplied, res_total_cost, cvar = charger_siting(
        comb_SOC, hexagons, hours, car_kWh, comb_demand_cars, comb_demand_kWh, comb_prices,
        comb_probability, P, charger_cost, opportunity_cost, alpha, beta)
    charger_hours_dict[length(hours)] = charger_in_hex
    energy_hours_dict[length(hours)] = value.(E_not_supplied)
    cost_hours_dict[length(hours)] = res_total_cost
    cvar_hours_dict[length(hours)] = cvar
end

In [ ]:
for hours in hours_lst
    println(length(hours), " hours: ",
        "charger_in_hex = ", sum(charger_hours_dict[length(hours)]),
        " / Energy not supplied = ", sum(energy_hours_dict[length(hours)]),
        " / total cost = ", cost_hours_dict[length(hours)],
        " / Cvar = ", cvar_hours_dict[length(hours)]
    )
end

In [ ]:
for (key, value) in charger_hours_dict
    CSV.write(
        string("sensitivity/hours/charger_in_hex_",string(key),".csv"),
        DataFrame(value)
    )
end
for (key, value) in energy_hours_dict
    CSV.write(
        string("sensitivity/hours/E_not_supplied_",string(key),".csv"),
        DataFrame(value)
    )
end
CSV.write(
        string("sensitivity/hours/cost_dict.csv"),
        cost_hours_dict
    )
CSV.write(
        string("sensitivity/hours/cvar_dict.csv"),
        cvar_hours_dict
    )

In [ ]:
println(opportunity_cost)
println(hours)

In [ ]:
op_cost_lst = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
charger_op_cost_dict = Dict()
energy_op_cost_dict = Dict()
cost_op_cost_dict = Dict()
cvar_op_cost_dict = Dict()
for opportunity_cost in op_cost_lst
    charger_in_hex, E_not_supplied, res_total_cost, cvar= charger_siting(
        comb_SOC, hexagons, hours, car_kWh, comb_demand_cars, comb_demand_kWh, comb_prices,
        comb_probability, P, charger_cost, opportunity_cost, alpha, beta)
    charger_op_cost_dict[opportunity_cost] = charger_in_hex
    energy_op_cost_dict[opportunity_cost] = value.(E_not_supplied)
    cost_op_cost_dict[opportunity_cost] = res_total_cost
    cvar_op_cost_dict[opportunity_cost] = cvar
end

In [ ]:
for opportunity_cost in op_cost_lst
    println(opportunity_cost, 
        " Opportunity Cost: ", "charger_in_hex = ", sum(charger_op_cost_dict[opportunity_cost]),
        " / Energy not supplied = ", sum(energy_op_cost_dict[opportunity_cost]),
        " / total cost = ", cost_op_cost_dict[opportunity_cost],
        " / Cvar = ", cvar_op_cost_dict[opportunity_cost]
    )
end

In [ ]:
for (key, value) in charger_op_cost_dict
    CSV.write(
        string("sensitivity/opportunity/charger_in_hex_",string(key),".csv"),
        DataFrame(value)
    )
end
for (key, value) in energy_op_cost_dict
    CSV.write(
        string("sensitivity/opportunity/E_not_supplied_",string(key),".csv"),
        DataFrame(value)
    )
end
CSV.write(
        string("sensitivity/opportunity/cost_dict.csv"),
        cost_op_cost_dict
    )
CSV.write(
        string("sensitivity/opportunity/cvar_dict.csv"),
        cvar_op_cost_dict
    )

In [ ]:
println(opportunity_cost)
println(hours)

In [ ]:
beta_lst = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
charger_beta_dict = Dict()
energy_beta_dict = Dict()
cost_beta_dict = Dict()
cvar_beta_dict = Dict()
for beta in beta_lst
    charger_in_hex, E_not_supplied, res_total_cost, cvar= charger_siting(
        comb_SOC, hexagons, hours, car_kWh, comb_demand_cars, comb_demand_kWh, comb_prices,
        comb_probability, P, charger_cost, opportunity_cost, alpha, beta)
    charger_beta_dict[beta] = charger_in_hex
    energy_beta_dict[beta] = value.(E_not_supplied)
    cost_beta_dict[beta] = res_total_cost
    cvar_beta_dict[beta] = cvar
end

In [ ]:
for beta in beta_lst
    println(beta, 
        " beta: ", "charger_in_hex = ", sum(charger_beta_dict[beta]),
        " / Energy not supplied = ", sum(energy_beta_dict[beta]),
        " / total cost = ", cost_beta_dict[beta],
        " / Cvar = ", cvar_beta_dict[beta]
    )
end

In [ ]:
for (key, value) in charger_beta_dict
    CSV.write(
        string("sensitivity/beta/charger_in_hex_",string(key),".csv"),
        DataFrame(value)
    )
end
for (key, value) in energy_beta_dict
    CSV.write(
        string("sensitivity/beta/E_not_supplied_",string(key),".csv"),
        DataFrame(value)
    )
end
CSV.write(
        string("sensitivity/beta/cost_dict.csv"),
        cost_beta_dict
    )
CSV.write(
        string("sensitivity/beta/cvar_dict.csv"),
        cvar_beta_dict
    )

charger_cost = [600, 400, 100]# investment cost for charger 3 kW, 7 kW und 11 kW in €/kW
charger_cost_delta = 600
charger_cost_dict = Dict()
energy_dict = Dict()
for i in range(0, step=100, stop=charger_cost_delta)
    charger_in_hex, x_charger, E_not_supplied, = charger_siting(
        SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, prices,
        probability, P, charger_cost.+i, opportunity_cost, alpha, beta)
    charger_cost_dict[charger_cost.+i] = charger_in_hex
    energy_dict[charger_cost.+i] = value.(E_not_supplied)
end

for key in keys(charger_cost_dict)
    println(key, " Charger Cost: ", "charger_in_hex = ", sum(charger_cost_dict[key]),
        " / Energy not supplied = ", sum(energy_dict[key]))
end